# Objetivos del Notebook

En el trabajo práctico anterior, hemos conversado de algunos principios para diseñar una API web, definiendo puntos para acceder a recursos. Hemos implementando una API web básica en Python con el framework Flask. Este notebook 5 tiene como objetivos:

- utilizar APIs web existentes

- manipular datos al formato JSON utilizando librerias Python para la Ciencia de Datos: Pandas, MatplotLib

- utilizar el SGBD no relacional Mongo describiendo primeras características que lo diferencia de MySQL

# 1. Utilizar la API OpenAQ

<a href="https://openaq.org">OpenAQ</a> es una iniciativa colaborativa ciudadana y científica que apunta a proveer datos abiertos y herramientas para monitorear y proteger la calidad del aire. Esta comunidad recopiló ~300 millones de medidas del aire en 67 paises.

En particular, OpenAQ toma la forma de:

- una API Web, desarrollada en Javascript (framework Node.js), la cual interactua con el SGBD relacional PostgreSQL. Todo el código está abierto bajo la licencia <i>MIT License</i>: <a href="https://github.com/openaq/openaq-api">OpenAQ-API</a>

- Una aplicación web, desarrollada en Javascript tambien que permite navegar visualmente en los datos: <a href="https://dolugen.github.io/openaq-browser/#/cities">OpenAQ-browser</a>

La documentación de la API se encuentra aqui: <a href="https://docs.openaq.org/">documentación</a>



<b>Pregunta</b>:

- ¿En algunas palabras cuáles son las características de la licencia MIT License? ¿Cuál es la diferencia principal con la licencia GPL?
- ¿Cuántos puntos de acceso tiene OpenAQ-API?
- ¿Cuántas tablas tiene la Base de Datos utilizada por OpenAQ?

Ejemplo de consulta a OpenAQ-API para responder a la pregunta siguiente:
<b>¿Cuál es el nivel de contaminación en la ciudad de Valdivia el 29 de octubre 2018?</b>

In [ ]:
# importar la librería Requests que sirve para enviar consultas HTTP
import requests 
  
# api-endpoint 
endpoint = "https://api.openaq.org/v1/measurements"
parameters = "?country=CL&city=Valdivia&include_fields=&limit=1000&date_from=2018-10-28&date_to=2018-10-30"
URL = endpoint+parameters

r = requests.get(url = URL) 
  
# extracting data in json format 
data = r.json() 
  
print(data)

El código anterior devuelve un solo objeto JSON bruto, en el cual se encuentra un campo <i>results</i>. El valor del campo <i>results</i> es un arreglo de objetos JSON que corresponden a medidas de la contaminación en Valdivia en distintas fechas.

Para acceder a valores particulares del JSON, la sintáxis es simple. Por ejemplo, si queremos ver el contenido del campo <i>meta</i>:

In [ ]:
data["meta"]

Observen cómo navegar en los distintos elementos del objeto JSON:

In [ ]:
data["meta"]["license"]

In [ ]:
data["results"][1]

In [ ]:
data["results"][1]["coordinates"]["latitude"]

En Python, la librería <a href="https://pandas.pydata.org/">Pandas</a> permite facilitar la manipulación de datos. A continuación, mostramos cómo cargar los datos de la API en una tabla (un "DataFrame" en la terminología de Pandas).

In [ ]:
import pandas as pd

valdivia_df = pd.DataFrame(data['results']) #dataframe(A)mostrar A en un tipo de tabla
#Mostramos los 10 primeros resultados
valdivia_df[0:10] #equivalente a: valdivia_df.head(10)

Las columnas <code>coordinates</code> y <code>date</code> contienen valor de tipo objeto JSON. Para evitar tener problema a continuación con estos valores, normalizamos el dataframe:

In [ ]:
#Fase de normalización que permite transformar un objeto JSON en columnas
from pandas.io.json import json_normalize
data_normalized=json_normalize(data['results'])

valdivia_df = pd.DataFrame(data_normalized)
#Mostramos los 10 primeros resultados
valdivia_df.head(10)

Como la clausula "SELECT" en SQL, Pandas permite selectionnar las columnas que se quiere mostrar:

In [ ]:
valdivia_df2=valdivia_df[['city','country','date.local','parameter','value']]
#Renombrar la columna "date.local" para simplificar
valdivia_df2 = valdivia_df2.rename(columns={'date.local': 'date'})
valdivia_df2.head(10)

Como en SQL, Pandas permite ordenar los resultados (el método <code>sort_values</code> es equivalente al <code>order by</code> en SQL). En nuestro ejemplo, queremos ordenar el resultado según la fecha.

In [ ]:
valdivia_df2.sort_values(['date'],ascending=True).head(10)

La API devuelve varias medidas cada hora, por un tema de simplicidad, vamos a generalizar los datos para tener solamente 1 valor por hora y por tipo de particulas. Como en SQL, Pandas tiene un método para agrupar valores. En nuestro ejemplo, agrupamos según el tipo de particula (<code>parameter</code>) y la fecha (<code>date</code>), y conservarmos el promedio de las medidas.

In [ ]:
import numpy as np

valdivia_df3=valdivia_df2.groupby(['parameter','date']).agg(np.average)
valdivia_df3=valdivia_df3.reset_index()
valdivia_df3.head(10)

Finalmente, Pandas tambien permite filtrar las filas de una tabla (equivalente del <code>WHERE</code> en SQL). En nuestro ejemplo, creamos 4 dataframes que contienen los datos por cada tipo de particula (co, pm10, pm25 y so2).

In [ ]:
valdivia_df_co=valdivia_df3[(valdivia_df3.parameter=='co')]
valdivia_df_pm10=valdivia_df3[(valdivia_df3.parameter=='pm10')]
valdivia_df_pm25=valdivia_df3[(valdivia_df3.parameter=='pm25')]
valdivia_df_so2=valdivia_df3[(valdivia_df3.parameter=='so2')]
valdivia_df_co.head()

Para más detalles sobre la API de Pandas, se puede consultar:

- <a href="https://www.dataquest.io/blog/pandas-cheat-sheet/">Pandas cheat sheet</a>

- <a href="https://pandas.pydata.org/pandas-docs/stable/">Documentación de Pandas</a>

Una vez que tenemos nuestros datos listos (o "preprocesados"), a menudo se necesita visualizar estos datos de alguna manera. En Python, existen varias librerías de visualización de datos, entre las cuales las más utilizadas son: MatplotLib, Plotly o Geoplotlib. Ejemplo de visualización realizada con Geoplotlib.

<img src="geoplotlib.png"></img>




Como primer enfoque, a continuación vamos simplemente visualizar un Line Chart mostrando la evolución de la cantidad de CO en Valdivia el día 28 de octubre:

In [ ]:
import matplotlib.pyplot as plt

#Definición del tamaño de la figura
plt.figure(figsize=(15,6))

#Definición de los datos que sirven de eje X e Y
x_co = valdivia_df_co['date']
y_co = valdivia_df_co['value']

#Leyenda
plt.xlabel('Date')
plt.ylabel('Cantidad de CO')
plt.title('Cantidad de CO en Valdivia el día 28 de octubre')
plt.xticks(rotation=90)

#Dibujar la visualización
plt.plot(x_co,y_co)

<b>Ejercicio:</b>

- a) Conectarse a la API OpenAQ para saber cuáles son los lugares en Chile dónde se recopila medidas de contaminación.

- b) Crear una visualización: Por ejemplo, mostrar en un histograma, cuántas mediciones hay por cada lugar en Chile.

- c) Opcional: Mostrar en un mapa de Chile, los lugares dónde se recopila medidades de contaminación.

In [ ]:
# a) 

import requests 
import pandas as pd
import numpy as np

# api-endpoint 
endpoint = "https://api.openaq.org/v1/cities"
parameters = "?country=CL"
URL = endpoint+parameters

r = requests.get(url = URL) 
  
# extracting data in json format 
data = r.json() 



valdivia_df = pd.DataFrame(data['results']) #dataframe(A)mostrar A en un tipo de tabla
valdivia_df=valdivia_df[['city','count']]
#valdivia_df = valdivia_df_g.count()

#Mostramos los 10 primeros resultados
valdivia_df[0:10] #equivalente a: valdivia_df.head(10)

print(valdivia_df)

In [ ]:
# b)
import matplotlib.pyplot as plt

#Definición del tamaño de la figura
plt.figure(figsize=(20,10))

#Definición de los datos que sirven de eje X e Y
x_co = valdivia_df['city']
y_co = valdivia_df['count']

#Leyenda
plt.xlabel('Ciudades')
plt.ylabel('Cantidad de mediciones')
plt.title('Mediciones en las ciudades de Chile')
plt.xticks(rotation=90)

#Dibujar la visualización
plt.bar(x_co,y_co)

# 2. Utilizar el SGBD no relacional Mongo 

MongoDB es un SGBD creado en 2009 (en 2018: version 4). Es un SGBD que no implementa el modelo Relacional (Tabla/Tupla/Claves, datos atómicos) como MySQL o PostgreSQL. La idea inicial de Mongo es ofrecer una solución simple para almacenar objetos JSON.

En Mongo, los conceptos claves son:
- los <b>Documentos</b>: un documento corresponde a objeto JSON.
- las <b>Colecciones</b>: una colección es un conjunto de documentos. Nota Bene: en una colección Mongo, los documentos pueden tener una estructura distinta.

En Mongo, no existen directamente los conceptos de Tabla, Tupla o Claves primerias/foraneas. Sin embargo, podemos ver una equivalencia entre Tabla y Colección, y entre Tupla y Documento.

Verificar que Mongo está instalado correctamente ejecutando:
- el programa principal del SGBD: <code>mongod &</code> desde un terminal
- el programa CLI (Command Line Interface): <code>mongo</code>

Vamos a interactuar con Mongo desde un entorno Python, por lo tanto utilizaremos el driver <code>PyMongo</code> (que instalar).


In [3]:
import pymongo
from pymongo import MongoClient

#Nos conectamos a la instancia de Mongo local, puerto 27017
client = MongoClient('localhost', 27017)

#Nos conectamos a la BD 'notebook5_db'
db = client['notebook5_db']

#Dentro de la BD, creamos una nueva colección llamada 'personas'
collection = db['personas']

A continuación, mostramos un ejemplo de como insertar nuevos documentos en una colección:

In [4]:
#Creamos un nuevo documento

aDocument = {"Nombre":"Juan", "Edad":23, "Asignaturas":["INFO261", "INFO267", "INFO268"] }
otherDocument = {"Nombre":"Maria", "Edad":24, "Asignaturas":["INFO261", "INFO267", "INFO268", "INFO331"] }

collection.insert_many([aDocument,otherDocument])

Un ejemplo de como consultar nuestra colección agregando un filtro para ver solamente los documentos que tienen un campo "Edad"=23.

In [ ]:
from pprint import pprint

for doc in collection.find({"Edad":23}):
    pprint(doc)

Para más detalles sobre la API de Mongo y PyMongo para hacer consultas:
- http://api.mongodb.com/python/current/tutorial.html
- https://docs.mongodb.com/manual/crud/

<b>Ejercicio</b>
- crear/borrar/modificar documentos desde Mongo CLI y desde PyMongo 
- realizar algunas consultas básicas

# 3. Trabajo práctico 1

1. En el marco de un proyecto "Smart City Valdivia", se desea almacenar mediciones de particulas en el aire en Valdivia a partir de la API OpenAQ. Estos datos se cruzaron a posteriori con datos meteorologicos de Valdivia (por ejemplo desde la API https://openweathermap.org/api). Diseñar un modelo Entidad-Asociación que permita almacenar esta información y extenderla a otras ciudades del sur de Chile.

2. Proponer un modelo de documentos JSON para almacenar está información en Mongo.

3. Conectarse a la API OpenAQ para empezar a llenar su base de datos Mongo.


# 4. Trabajo práctico 2

1. Conectarse a la API de Twitter.

2. Recopilar una lista de usuarios que utilizan el hashtag #ValdiviaCL en sus tweets.




In [ ]:
# 3) a)
- Diccionario de datos:
    nombre_ciudad
    latitud
    longitud
    estacion
    pm25
    pm10
    hora
    fecha
    velocidad_viento
    temperatura
    precipitacion
    
- Entidad-Asociacion:
              Ciudad -----1,n------<ubicarse> -------1,1---Estacion
             (nombre)                                     (nombre,latitud,longitud)
                 |                                          |
                 |                                          | 
                 1,n                                        0,n
                <tener>(fecha)                              | 
                 |                                          <medir>(hora,fecha)      
                 |                                          |
                1,1                                         1,1  
                 |                                          |
                 |                                          |
                Meteorologia                                Medicion
                (temperatura,viento,precipitacion)             (pm25,pm10)
                

                
# b) 
    ESTACION
{
  "nombre": "...",
  "coordinates": {
      "latitude": 43.34,
      "longitude": 23.04
  }
}

    METEO
{
  "temperatura": 12.6,
  "viento": "",
  "precipitacion": 2.0,
  "ciudad":
      {
          "nombre":"Valdivia"  
      }
}      

In [5]:
# c)

import pymongo
from pymongo import MongoClient

#Nos conectamos a la instancia de Mongo local, puerto 27017
client = MongoClient('localhost', 27017)

#Nos conectamos a la BD 'notebook5_db'
db = client['notebook5_db']

#Dentro de la BD, creamos una nueva colección llamada 'autos'
collection = db['autos']


Nissan = {"Modelo":"Nissan", "Año":2013, "Matricula": "GH-FT-75" }
BMW = {"Modelo":"BMW-GTR3", "Año":2016, "Matricula": "YG-PL-43"}

collection.insert_many([Nissan,BMW])

In [6]:
# 2

import tweepy
from tweepy.auth import OAuthHandler
import pandas as pd

access_token = '1060687993462288384-4VhejwTPXIuONV1SrExxRveBpmmGo3'
access_token_secret = 'ATBHBlUznUajJzrsd9wJ2LnyvySanFbqmtxrJ1Vsd1uO0'
consumer_key = 'qjXCJ8FakSf8D7uLwNeO6uPp1'
consumer_secret = 'vCbte8OzLs3NXnuq6gwbzi4l9FMbljC01fZu7BkT5i0tyEJ4db'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
for tweet in tweepy.Cursor(api.search,q="ValdiviaCL",count=10,lang="es",
since="2017-04-03").items(10): print(tweet.user.screen_name)

ImportError: No module named twitter